In [ ]:
import numpy as np
import random
from collections import defaultdict
from scipy.optimize import minimize
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Define the GridWorld environment
class GridWorld:
    def __init__(self, size=5):
        self.size = size  # Grid size (5x5)
        self.features = np.zeros((size, size))
        self.payoffs = {}
        self.define_features()
        self.states = [(x, y) for x in range(size) for y in range(size)]
    
    def define_features(self):
        # Define features and their payoffs
        # Feature 1: Payoff = 1
        # Feature 2: Payoff = 0.95
        # Feature 3: Payoff = -1
        for x in range(self.size):
            for y in range(self.size):
                if (x + y) % 3 == 0:
                    self.features[x, y] = 1  # Feature 1
                elif (x + y) % 3 == 1:
                    self.features[x, y] = 2  # Feature 2
                else:
                    self.features[x, y] = 3  # Feature 3
        self.payoffs = {1: 1, 2: 0.95, 3: -1}
    
    def get_payoff(self, position):
        x, y = position
        feature = self.features[x, y]
        return self.payoffs[feature]

# Define the Agent class
class Agent:
    def __init__(self, gridworld, discount_factor=0.95):
        self.gridworld = gridworld
        self.discount_factor = discount_factor
        self.actions = ['up', 'down', 'left', 'right', 'stay']
        self.num_actions = len(self.actions)
        self.states = gridworld.states
        self.state_indices = {state: idx for idx, state in enumerate(self.states)}
        self.num_states = len(self.states)
    
    def move(self, position, action):
        x, y = position
        if action == 'up' and x > 0:
            x -= 1
        elif action == 'down' and x < self.gridworld.size - 1:
            x += 1
        elif action == 'left' and y > 0:
            y -= 1
        elif action == 'right' and y < self.gridworld.size - 1:
            y += 1
        # 'stay' or invalid moves result in no change
        return (x, y)
    
    def value_iteration(self, max_iter=1000, tol=1e-7):
        V = np.zeros(self.num_states)
        for iteration in range(max_iter):
            V_prev = V.copy()
            for idx, state in enumerate(self.states):
                action_values = []
                for action in self.actions:
                    new_state = self.move(state, action)
                    reward = self.gridworld.get_payoff(new_state)
                    new_state_idx = self.state_indices[new_state]
                    Q_value = reward + self.discount_factor * V_prev[new_state_idx]
                    action_values.append(Q_value)
                # Correctly update V(s) using the Bellman equation
                V[idx] = logsumexp(action_values)
            # Check for convergence
            if np.max(np.abs(V - V_prev)) < tol:
                break
        return V


    
    def get_action_probabilities(self, state, V):
        action_values = []
        for action in self.actions:
            new_state = self.move(state, action)
            reward = self.gridworld.get_payoff(new_state)
            new_state_idx = self.state_indices[new_state]
            Q_value = reward + self.discount_factor * V[new_state_idx]
            action_values.append(Q_value)
        # Compute choice probabilities using softmax
        log_probs = action_values - logsumexp(action_values)
        probs = np.exp(log_probs)
        return probs

# Function to generate a trajectory based on the process type
def generate_trajectory(agent, start_position, process_type='preference_shocks', max_steps=10):
    trajectory = []
    position = start_position
    # Compute the true value function using true parameters
    agent.gridworld.payoffs = {1: 1, 2: 0.95, 3: -1}
    V = agent.value_iteration()
    for step in range(max_steps):
        probs = agent.get_action_probabilities(position, V)
        action = np.random.choice(agent.actions, p=probs)
        new_position = agent.move(position, action)
        reward = agent.gridworld.get_payoff(new_position)
        trajectory.append((position, action, reward))
        position = new_position
    return trajectory


# Generate expert data (nt trajectories)
def generate_expert_data(agent, nt):
    trajectories = []
    grid_size = agent.gridworld.size
    # Set the gridworld payoffs to true parameters
    agent.gridworld.payoffs = {1: 1, 2: 0.95, 3: -1}
    V_true = agent.value_iteration()
    for _ in range(nt):
        start_position = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
        trajectory = []
        position = start_position
        for step in range(10):  # Assuming max_steps=10
            probs = agent.get_action_probabilities(position, V_true)
            action = np.random.choice(agent.actions, p=probs)
            new_position = agent.move(position, action)
            reward = agent.gridworld.get_payoff(new_position)
            trajectory.append((position, action, reward))
            position = new_position
        trajectories.append(trajectory)
    return trajectories



# NFXP Algorithm with corrected likelihood computation
def nfxp_log_likelihood(params, agent, trajectories):
    # Set gridworld payoffs based on parameters
    agent.gridworld.payoffs = {1: params[0], 2: params[1], 3: params[2]}
    # Solve for value function V
    V = agent.value_iteration()
    # Compute log-likelihood
    log_likelihood = 0
    for trajectory in trajectories:
        for (position, action, reward) in trajectory:
            probs = agent.get_action_probabilities(position, V)
            action_index = agent.actions.index(action)
            # Add a small epsilon to prevent log(0)
            log_likelihood += np.log(probs[action_index] + 1e-12)
    return -log_likelihood  # Negative log-likelihood for minimization



def estimate_nfxp(agent, trajectories):
    initial_params = [0.5, 0.5, -0.5]  # Alternative initial parameters
    bounds = [(-2, 2), (-2, 2), (-2, 2)]  # Adjusted bounds
    result = minimize(
        nfxp_log_likelihood,
        initial_params,
        args=(agent, trajectories),
        bounds=bounds,
        method='L-BFGS-B',
        options={'maxiter': 1000, 'disp': False}
    )
    estimated_params = result.x
    return estimated_params


# Run experiments
def run_experiment(nt_list=[10, 15, 20], num_trials=20):
    gridworld = GridWorld()
    agent = Agent(gridworld)
    results = defaultdict(list)
    for nt in nt_list:
        for trial in range(num_trials):
            trajectories = generate_expert_data(agent, nt)
            # NFXP estimation
            nfxp_estimates = estimate_nfxp(agent, trajectories)
            results['nfxp'].append((nt, trial, 'preference_shocks', nfxp_estimates))
            print(f"Completed nt={nt}, trial={trial+1}/{num_trials}")
    return results


# Function to compute mean estimates and errors
def compute_mean_estimates(df, algorithm_name):
    # True parameter values
    true_params = np.array([1, 0.95, -1])
    # Process preference_shocks
    pref_shocks_df = df[df['process_type'] == 'preference_shocks']
    mean_estimates_pref = pref_shocks_df.groupby('nt')[['Feature1', 'Feature2', 'Feature3']].mean()
    mean_estimates_pref['Process_Type'] = 'Preference Shocks'
    # Compute errors
    for i, feature in enumerate(['Feature1', 'Feature2', 'Feature3']):
        mean_estimates_pref[f'Error_{feature}'] = mean_estimates_pref[feature] - true_params[i]
    print(f"\n{algorithm_name} - Mean Estimated Parameters under Preference Shocks:")
    print(mean_estimates_pref[['Feature1', 'Feature2', 'Feature3']])
    print(f"\n{algorithm_name} - Estimation Errors under Preference Shocks:")
    print(mean_estimates_pref[[f'Error_Feature1', f'Error_Feature2', f'Error_Feature3']])
    
    # Plotting
    for feature in ['Feature1', 'Feature2', 'Feature3']:
        plt.figure(figsize=(10, 6))
        subset = mean_estimates_pref
        plt.plot(subset.index, subset[feature], marker='o', label='Preference Shocks')
        plt.title(f'{algorithm_name} - Mean Estimated {feature}')
        plt.xlabel('Number of Trajectories (nt)')
        plt.ylabel(f'Mean Estimated {feature}')
        plt.legend()
        plt.grid(True)
        plt.show()

# Main execution
if __name__ == "__main__":
    results = run_experiment()
    
    # Access NFXP estimates
    nfxp_results = results['nfxp']
    nfxp_df = pd.DataFrame(nfxp_results, columns=['nt', 'trial', 'process_type', 'estimates'])
    nfxp_df[['Feature1', 'Feature2', 'Feature3']] = pd.DataFrame(nfxp_df['estimates'].tolist(), index=nfxp_df.index)
    nfxp_df.drop('estimates', axis=1, inplace=True)
    
    # Compute and display mean estimates for NFXP
    compute_mean_estimates(nfxp_df, 'NFXP')